# Finetuning von Modellen
Nehmen wir an, Sie haben jetzt eine Liste mit vielversprechenden Modellen in der engeren Wahl. Sie müssen sie nun noch verfeinern. Sehen wir uns einige Möglichkeiten an, wie Sie das tun können.

# Grid Search
Eine Möglichkeit ist, mit den Hyperparametern (Konfigurationsparamter der ML-Algorithmen) manuell zu spielen, bis man eine hinreichend gute Kombination von Hyperparameterwerten findet. Das wäre sehr mühsam und Sie haben vielleicht nicht die Zeit, viele Kombinationen auszuprobieren. Stattdessen sollten Sie Scikit-Learn's **GridSearchCV** für die Suche nach geeigneten Hyperparametern einsetzen. Sie brauchen der Funktion nur zu sagen, mit welchen Hyperparametern sie experimentieren soll und welche Werte ausprobiert werden wollen. GridSearchCV wird dann alle möglichen Kombinationen von Hyperparameterwerten unter Verwendung von Kreuzvalidierung bewerten. Der folgende Code sucht z.B. nach der besten Kombination von Hyperparameterwerten für den RandomForestRegressor:

In [1]:
from sklearn.model_selection import GridSearchCV

# Array mit allen Paramtern, die im Rahmen des Trainings verändert werden sollen
param_grid = [
    {'n_estimators': [3, 10, 30],'max_features': [2, 4, 6, 8]},
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
]

forest_reg = RandomForestRegressor()
grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
scoring='neg_mean_squared_error')
grid_search.fit(housing_prepared, housing_labels)

# Ausgabe der besten Paramter:
grid_search.best_params_

grid_search.best_estimator_

# Alle Scores der Leistungsmessungen
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

NameError: name 'RandomForestRegressor' is not defined

Durch GridSearchCV werden unterschiedliche Paramter für den RandomForest-Algorithmus ausprobiert. Im ersten Schritt werden alle Kombinationen aus n_estimators und max_features (12 Kombinationen) ausprobiert. In einem zweiten Durchgang werden dann nochmal ohne bootstrapping zwei n_estiamtors mit drei max_features ausprobiert. Zusätzlich wird 5-fache Crossvalidierung ausprobiert. In Summe werden also 12+6 = 18 Kombinationen in 5 facher Cross-Validierung ausprobiert. Das macht in Summe 90 Trainingsrunden, um die besten Paramter zu ermitteln. Das dauert seine Zeit und sollte nur auf einem leistungsstarken Rechner oder über Nacht ausgeführt werden.

# Zufällige Suche
Der Ansatz der Rastersuche (GridSearchCV) ist in Ordnung, wenn Sie relativ wenige Kombinationen ausprobieren,
wie im vorherigen Beispiel. Aber wenn der Hyperparameter-Suchraum groß ist, sollte man stattdessen **RandomizedSearchCV** verwenden. Diese Klasse arbeitet auf die gleiche Weise wie die Klasse GridSearchCV, aber anstatt alle möglichen Kombinationen auszuprobiere, bewertet sie eine bestimmte Anzahl von Zufallskombinationen durch Auswahl von zufälligen Werten für jeden Hyperparameter bei jeder Iteration. Dieser Ansatz hat zwei Hauptvorteile: 
- Wenn Sie die randomisierte Suche z.B. über 1.000 Iterationen laufen lassen, wird dieser Ansatz 1.000 verschiedene Werte für jeden Hyperparameter untersuchen (statt nur ein paar Werte pro Hyperparameter mit dem Grid-Search-Ansatz).
- Sie haben mehr Kontrolle über das Rechenbudget, das Sie den Hyperparametern zuweisen wollen. Sie können einfach die Anzahl der Iterationen festlegen

# Evaluation des Modells auf dem Testdatensatz
Nachdem Sie eine Zeit lang an Ihren Modellen gefeilt haben, verfügen Sie letztlich über ein Modell, das ausreichend gut funktioniert. Jetzt ist es an der Zeit, das endgültige Modell auf dem Testset zu evaluieren. Es gibt nichts Besonderes an diesem Prozess; holen Sie einfach die Prädiktoren und die Labels aus Ihrem Testsatz, führen Sie Ihre Datentransformationspipeline (full_pipeline) aus, um die Daten zu transformieren (rufen Sie transform() auf, nicht fit_transform()!), und evaluieren Sie das endgültige Modell auf dem Testsatz. Der folgende Code ist aus Performance-gründen im Notebook nicht in vertretbarer Zeit ausführbar. Es gibt aber eine separate Übungsaufgabe zu Mashine Learning.

In [ ]:
#Hohe das Modell mit den besten Hyperparametern
final_model = grid_search.best_estimator_

#Trenne den Testdatensatz in Faktoren (x) und Labels (y)
X_test = strat_test_set.drop("median_house_value", axis=1)
y_test = strat_test_set["median_house_value"].copy()

#Transforamtion der Testdaten (z.B. Behandlung von kategoriellen Werten)
X_test_prepared = full_pipeline.transform(X_test)

# Vorhersageberechnung
final_predictions = final_model.predict(X_test_prepared)

# Bewertung der Leistungsfähigkeit bzw. Prognosegüte des Modells
final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse) 